Partially based on this tutorial: https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-1-fd544fab149

In [2]:
import tensorflow as tf
import numpy as np

In [56]:
n = 5

def pullBandit(i,p):
    #return 1 with probability p, 0 otherwise
    if np.random.rand(1) < p[i]:
        return 1
    else:
        return -1

In [13]:
tf.reset_default_graph()

#Feed-forward part of the network
weights = tf.Variable(tf.ones([n]))
chosen_action = tf.argmax(weights,0)

#training procedure
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [67]:
n_episodes = 1000
total_reward = np.zeros(n)
epsilon = 0.1

def main(epsilon, n_runs):
    n_correct = 0
    for j in range(n_runs):
        #Probabilities that any given bandit yield a reward
        p = np.random.rand(n)
        init = tf.global_variables_initializer();
        with tf.Session() as sess:
            sess.run(init)
            i = 0
            while i < n_episodes:
                #Choose either a random action or one from our network.
                if np.random.rand(1) < epsilon(i):
                    action = np.random.randint(n)
                else:
                    action = sess.run(chosen_action)

                reward = pullBandit(action,p)

                #Update the network.
                _,resp,ww = sess.run([update,responsible_weight,weights], 
                                     feed_dict={reward_holder:[reward],
                                                action_holder:[action]})

                #Update our running tally of scores.
                total_reward[action] += reward
                i+=1
        if np.argmax(ww) == np.argmax(np.array(p)):
            n_correct+=1
    print("The agent identifies the correct bandit in " + str(n_correct) +" out of "+str(n_runs)+" runs")

In [73]:
#Calling main with this uses a constant epsilon (the most naive approach)
def const_epsilon(i):
    return 0.1

print("The naive approach with a constant epsilon leads to fairly bad results:")
main(const_epsilon,100)
      
def decaying_epsilon(i):
    initial_eps = 0.5
    lam = 0.005
    return initial_eps*np.exp(-lam*i)
      
print("With a larger epsilon and decrease over time:")
main(decaying_epsilon,100)

The naive approach with a constant epsilon leads to fairly bad results:
The agent identifies the correct bandit in 61 out of 100 runs
With a larger epsilon and decrease over time:
The agent identifies the correct bandit in 63 out of 100 runs


Takeaway: This doesn't seem to help much. Simple gradient descent presumably isn't good at handling the exploration/exploitation tradeoff.